# **Installing the dependencies**

In [ ]:
#https://github.com/graykode/nlp-tutorial/blob/master/5-1.Transformer/Transformer.py#L45

In [ ]:
!pip install transformers torch

In [ ]:
!pip install -U SentencePiece

In [ ]:
!pip install accelerate -U

In [ ]:
pip install datasets


# **Importing the dependencies**

In [ ]:
import json
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification, T5Tokenizer, T5ForSequenceClassification,TrainingArguments, Trainer
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import random
from datasets import Dataset
import numpy as np
import math
import pandas as pd
from collections import Counter
import sys
import json
# visualization libraries
import matplotlib.pyplot as plt
import numpy as np

# pytorch libraries
import torch # the main pytorch library
import torch.nn as nn # the sub-library containing Softmax, Module and other useful functions
import torch.optim as optim # the sub-library containing the common optimizers (SGD, Adam, etc.)

# huggingface's transformers library
#from transformers import RobertaForTokenClassification, RobertaTokenizer
from transformers import RobertaForTokenClassification, RobertaTokenizer
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer

# huggingface's datasets library
from datasets import load_dataset

# the tqdm library used to show the iteration progress
import tqdm
tqdmn = tqdm.notebook.tqdm#tqdm.tqdm# tqdm.notebook.tqdm

from datasets import load_dataset
import os
from transformers import get_scheduler
from datasets import Dataset
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import copy


In [ ]:
# loading surprise.solution dataset
with open('/home/naive123/nlp/Sumit/massive/suprise/surprise_data/surprise.solution', 'r') as solution_file:
    solutions = [json.loads(line) for line in solution_file] # converting string to object using json.loads

# **loading and perform preprocessing on blind dataset(Testing dataset)**

In [ ]:
solution_path='/home/naive123/nlp/Sumit/massive/suprise/massive_test.data'
with open(solution_path,'r') as file:
	files=file.readlines()
# files
# print(files)
# print(len(files))
# print('\n')
# print(files[0])
test_list=[]
for line in files:
	line=line.strip('\n')
	line=json.loads(line)
	test_list.append(line)
# # print(dict)
# print(len(files_list))
# # for element in dict:
# 	# print(element)
# 	# print('\n')

In [ ]:
test_list[:20]

[{'indoml_id': 1,
  'utt': 'Kindly show me my entire transaction record from the beginning of this year.'},
 {'indoml_id': 2, 'utt': 'Could you help me get this thing delivered today?'},
 {'indoml_id': 3, 'utt': 'Can you join forces with my mobile?'},
 {'indoml_id': 4,
  'utt': 'Can I get an update on the interest rate for my TD Bank savings account?'},
 {'indoml_id': 5, 'utt': "Absolutely! I'd love some more coffee too."},
 {'indoml_id': 6,
  'utt': 'What are the necessary tools required to jump-start my vehicle?'},
 {'indoml_id': 7,
  'utt': 'how long does it take for my credit score to recover from a bankruptcy filing?'},
 {'indoml_id': 8, 'utt': 'Could you please play the next track?'},
 {'indoml_id': 9,
  'utt': 'How do I go about increasing my credit limit on my Discover it cashback card?'},
 {'indoml_id': 10,
  'utt': 'Kindly provide me with the exact date of expiration of my Capital One card.'},
 {'indoml_id': 11,
  'utt': 'may I ask if you would prefer to communicate in swahil

In [ ]:
test_data=[]
for data in test_list:
    test_data.append(data['utt'])

In [ ]:
test_data

['Kindly show me my entire transaction record from the beginning of this year.',
 'Could you help me get this thing delivered today?',
 'Can you join forces with my mobile?',
 'Can I get an update on the interest rate for my TD Bank savings account?',
 "Absolutely! I'd love some more coffee too.",
 'What are the necessary tools required to jump-start my vehicle?',
 'how long does it take for my credit score to recover from a bankruptcy filing?',
 'Could you please play the next track?',
 'How do I go about increasing my credit limit on my Discover it cashback card?',
 'Kindly provide me with the exact date of expiration of my Capital One card.',
 'may I ask if you would prefer to communicate in swahili?',
 'What are the deadlines for filing my state and federal taxes this year?',
 "Today's fun fact: The Great Barrier Reef is so big that it can be seen from space! It spans over 2,300 kilometers (1,400 miles).",
 'How can I ensure that my paycheck is deposited directly into my account ev

In [ ]:
labels_list=[]
for label in solutions:
    labels_list.append(label['intent'])
unique_labels_list=[]
for x in labels_list:
    if x not in unique_labels_list:
        unique_labels_list.append(x)
# unique_labels_list, len(unique_labels_list)

label2id={}
id2label={}
for i, label in enumerate(unique_labels_list):
    label2id[label]=i
    id2label[i]=label

In [ ]:
test_labels=[0]*6000 # array of 6000 size having all test labels to be zero
test_int_labels=np.array(test_labels)

In [ ]:
test_dict={'utt':test_data, "intent":test_int_labels}
test_dict

{'utt': ['Kindly show me my entire transaction record from the beginning of this year.',
  'Could you help me get this thing delivered today?',
  'Can you join forces with my mobile?',
  'Can I get an update on the interest rate for my TD Bank savings account?',
  "Absolutely! I'd love some more coffee too.",
  'What are the necessary tools required to jump-start my vehicle?',
  'how long does it take for my credit score to recover from a bankruptcy filing?',
  'Could you please play the next track?',
  'How do I go about increasing my credit limit on my Discover it cashback card?',
  'Kindly provide me with the exact date of expiration of my Capital One card.',
  'may I ask if you would prefer to communicate in swahili?',
  'What are the deadlines for filing my state and federal taxes this year?',
  "Today's fun fact: The Great Barrier Reef is so big that it can be seen from space! It spans over 2,300 kilometers (1,400 miles).",
  'How can I ensure that my paycheck is deposited direct

In [ ]:
print(list(id2label.items())[:5])
print('\n')
print(list(label2id.items())[:5])


[(0, 'accept reservations'), (1, 'account blocked'), (2, 'alarm'), (3, 'application status'), (4, 'apr')]


[('accept reservations', 0), ('account blocked', 1), ('alarm', 2), ('application status', 3), ('apr', 4)]


In [ ]:
seed=50
torch.manual_seed(seed)
num_labels = 150
# model_max_length = 64
lr=6e-4
n_epochs = 30
batch_size= 32
min_batch_size=2
batch_size_c = batch_size / min_batch_size
early_stoping_patience = 6 # if valid loss will increase 3 consecutive times with a margin then training will stop.
# initial_model = "xlm-roberta-large"

# **Loading the pretrained intent classification Alexa XLMRoberta model and the tokenizer**

In [ ]:
# Define the BERT model and tokenizer
# keep `ignore_mismatched_sizes=True` so that the classification layer is randomly initialized
# model_name = "cartesinus/bert-base-uncased-amazon-massive-intent"  # Example: You can replace this with the specific RoBERTa variant you want to use
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification


# model_name = 'qanastek/XLMRoberta-Alexa-Intents-Classification'
# model_name="philschmid/habana-xlm-r-large-amazon-massive" # 91% accuracy on Intent classification base amazon massive
model_name="ibm/roberta-large-vira-intents"
# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)



In [ ]:
# Initialize the model
num_classes=150
model =RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True,output_hidden_states=True,output_attentions=True)
"""since model have its own id2label mapping and vice versa so, converting them from our id2label and label2id mapping as defined earlier"""
model.label2id=label2id
model.id2label=id2label


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ibm/roberta-large-vira-intents and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([181, 1024]) in the checkpoint and torch.Size([150, 1024]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([181]) in the checkpoint and torch.Size([150]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def add_encodings(example):
  tokenized_inputs = tokenizer(   example['utt'], truncation=True,  is_split_into_words=True)

  # print(tokenized_inputs)
  encodings = tokenizer(example["utt"], truncation=True, padding='max_length', is_split_into_words=True)
  # print(encodings)
  intent=example["intent"]

  return { **encodings, 'labels': intent }

In [ ]:
# train_dataset["attention_mask"].dtype

In [ ]:
test_dataset =  Dataset.from_dict(test_dict)
test_labels = test_dataset['intent']


test_dataset = test_dataset.map(add_encodings)#,

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
test_dataset

Dataset({
    features: ['utt', 'intent', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 6000
})

In [ ]:
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=min_batch_size)

In [ ]:
# Number of batches for each data

print(len(test_data))
print(test_data)

3000


In [ ]:
#check cuda or cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
n_layers=1
d_model = 1024
n_heads=1
d_k=512
d_v=512
d_ff=512
batch_size_1=min_batch_size
hidden_dropout_prob=0.1
num_labels=150

class GELU(nn.Module):

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

class Embedding(nn.Module):
   def __init__(self):
       super(Embedding, self).__init__()
       self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
       self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
       self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
       self.norm = nn.LayerNorm(d_model)

   def forward(self, x, seg):
       seq_len = x.size(1)
       pos = torch.arange(seq_len, dtype=torch.long)
       pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
       embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
       return self.norm(embedding)

class MultiHeadAttention(nn.Module):
   def __init__(self):
       super(MultiHeadAttention, self).__init__()
       self.W_Q = nn.Linear(d_model, d_k * n_heads)
       self.W_K = nn.Linear(d_model, d_k * n_heads)
       self.W_V = nn.Linear(d_model, d_v * n_heads)
       self.linear_layer = nn.Linear(n_heads * d_v, d_model)
       self.layer_norm = nn.LayerNorm(d_model)

   def forward(self, Q, K, V, attn_mask):
       # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
       residual, batch_size = Q, Q.size(0)
       # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
       q_s = self.W_Q(Q).view(batch_size_1, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
       k_s = self.W_K(K).view(batch_size_1, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
       v_s = self.W_V(V).view(batch_size_1, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]
      #  print(attn_mask.shape)
       attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

       # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
       context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
       context = context.transpose(1, 2).contiguous().view(batch_size_1, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
#        print(context.device)

       output = self.linear_layer(context)


       return self.layer_norm (output + residual), attn # output: [batch_size x len_q x d_model]

class PoswiseFeedForwardNet(nn.Module):

    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.l1 = nn.Linear(d_model, d_ff)
        self.l2 = nn.Linear(d_ff, d_model)

        self.relu = GELU()
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, inputs):
        residual = inputs
        output = self.l1(inputs)
        output = self.relu(output)
        output = self.l2(output)
        return self.layer_norm(output + residual)

class ScaledDotProductAttention(nn.Module):
   def __init__(self):
       super(ScaledDotProductAttention, self).__init__()

   def forward(self, Q, K, V, attn_mask):
      #  print(Q.shape)
      #  print(K.shape)
      #  print(V.shape)
      #  print(attn_mask.shape)
       scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
      #  print(scores.shape)
      #  print(attn_mask.shape)
       attn_mask = attn_mask.bool()
      #  print(attn_mask.shape)
       scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.

       attn = nn.Softmax(dim=-1)(scores)
       context = torch.matmul(attn, V)
       return  context, attn

class ClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self):
        super(ClassificationHead,self).__init__()
        self.dense = nn.Linear(d_model,d_model)#(config.hidden_size, config.hidden_size)
        classifier_dropout = hidden_dropout_prob       #( config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob     )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(d_model,num_labels)#(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet(d_model=d_model, d_ff=d_ff)
        self.classifier = ClassificationHead()
        self.loss_fct = nn.CrossEntropyLoss()

    def forward(self, enc_inputs, enc_self_attn_mask,labels):
        # print("enter")
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        # print("enc_outputs, attn ")
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        # print("enc_outputs ffn")
        logits = self.classifier(enc_outputs)
        # print("logits")
        loss = self.loss_fct(logits.view(-1, num_labels), labels.view(-1))
        # print("loss")
        return enc_outputs, attn,logits, loss

In [ ]:
encoder_layer = EncoderLayer()

In [ ]:
encoder_layer.to(device)


EncoderLayer(
  (enc_self_attn): MultiHeadAttention(
    (W_Q): Linear(in_features=1024, out_features=512, bias=True)
    (W_K): Linear(in_features=1024, out_features=512, bias=True)
    (W_V): Linear(in_features=1024, out_features=512, bias=True)
    (linear_layer): Linear(in_features=512, out_features=1024, bias=True)
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (pos_ffn): PoswiseFeedForwardNet(
    (l1): Linear(in_features=1024, out_features=512, bias=True)
    (l2): Linear(in_features=512, out_features=1024, bias=True)
    (relu): GELU()
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (classifier): ClassificationHead(
    (dense): Linear(in_features=1024, out_features=1024, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (out_proj): Linear(in_features=1024, out_features=150, bias=True)
  )
  (loss_fct): CrossEntropyLoss()
)

In [ ]:
# Prediction
def  prediction(best_model_path,best_encoder_path, eval_data):

    predicted=[]
    tokanised_exam=[]
    true_labels=[]

    predicted_for_classification_report=[]

    ##################################################
    #load best model


    print(best_model_path)

    model = RobertaForSequenceClassification.from_pretrained(best_model_path)
    model = model.eval().to(device)

    encoder_layer =EncoderLayer()

    encoder_layer=encoder_layer.to(device)
    encoder_layer.load_state_dict(torch.load(best_encoder_path))
    encoder_layer.eval()
    print("model device")
    # batch the train data so that each batch contains 4 examples (using 'batch_size')
    #test_data = torch.utils.data.DataLoader(test_dataset, batch_size=16)

    eval_data = eval_data



    # iterate through each batch of the eval data
    for i, batch in enumerate(tqdmn(eval_data)):



       # move the batch tensors to the same device as the
        batch = { k: v.to(device) for k, v in batch.items() }
        # send 'input_ids', 'attention_mask' and 'labels' to the model
        batch["attention_mask"]=batch["attention_mask"].unsqueeze(1)
#         print(len(batch["attention_mask"]))
        batch["attention_mask"] = batch["attention_mask"].expand(len(batch["attention_mask"]), d_k, d_k).to(device)

        if len(batch["labels"]) == min_batch_size :
            outputs = model(**batch)
            print(outputs.hidden_states[24].shape)
            # the outputs are of shape (loss, logits)
            #print(outputs[1])
    #         print("*")
    #         print(batch["attention_mask"].shape)
    #         print(len(batch["attention_mask"]))
    #         print(len(batch["input_ids"]))
    #         print(len(batch["labels"]))

            enc_outputs, attn, logits, loss = encoder_layer(outputs.hidden_states[24],batch["attention_mask"] , batch["labels"])#**batch outputs.attentions[23]
#             print(loss)

        print(logits.shape)
        # iterate through the examples

        # Apply softmax to get class probabilities
#         probabilities = torch.softmax(logits, dim=1)

        # Get the predicted class (index with the highest probability)
        pred_values = torch.argmax(logits, dim=1)
        pred_values = pred_values.tolist()
        predicted.append(pred_values)

        true_values =test_int_labels
        print(true_values.shape)
        true_labels.extend(true_values.tolist())

        predicted_for_classification_report.extend(pred_values)


          # go through all true and predicted values and store them in the confusion matrix
          # for true, pred in zip(true_values, pred_values):
          #     confusion[true.item()][pred.item()] += 1
    print(predicted_for_classification_report)
    return predicted, predicted_for_classification_report


In [ ]:
best_encoder_path="/home/naive123/nlp/Sumit/massive/model_1_extra_layer/roberta_ibm0.0001_32_30seed34/best_e_enl9entire_model.pth"
best_model_path="/home/naive123/nlp/Sumit/massive/model_1_extra_layer/roberta_ibm0.0001_32_30seed34/best_e_9"

In [ ]:
predicted, predicted_for_classification_report = prediction(best_model_path, best_encoder_path,test_data)

/home/naive123/nlp/Sumit/massive/model_1_extra_layer/roberta_ibm0.0001_32_30seed34/best_e_9
model device


  0%|          | 0/3000 [00:00<?, ?it/s]

torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])
(6000,)
torch.Size([2, 512, 1024])
torch.Size([2, 150])


KeyboardInterrupt: 

In [ ]:
predicted_labels=[id2label[i] for i in predicted_for_classification_report]
predicted_labels

['carry on',
 'lost luggage',
 'meeting schedule',
 'plug type',
 'w2',
 'tell joke',
 'application status',
 'car rental',
 'replacement card duration',
 'pto request status',
 'translate',
 'routing',
 'oil change when',
 'goodbye',
 'payday',
 'measurement conversion',
 'oil change when',
 'sync device',
 'new card',
 'todo list update',
 'do you have pets',
 'flip coin',
 'rewards balance',
 'spelling',
 'change speed',
 'date',
 'mpg',
 'what song',
 'change language',
 'redeem rewards',
 'spending history',
 'update playlist',
 'what song',
 'pin change',
 'how old are you',
 'redeem rewards',
 'change accent',
 'definition',
 'spelling',
 'mpg',
 'travel notification',
 'next song',
 'meal suggestion',
 'lost luggage',
 'restaurant reviews',
 'who do you work for',
 'book flight',
 'vaccines',
 'todo list update',
 'change volume',
 'change ai name',
 'international fees',
 'next holiday',
 'balance',
 'bill due',
 'smart home',
 'pto used',
 'change speed',
 'directions',
 'dam

In [ ]:
# len(predicted_for_classification_report)
# # target_names = labels_list
# from sklearn.metrics import classification_report
# print(classification_report(test_labels, predicted_labels[:-1]))#,labels=[i for i in range(150)]))


In [ ]:
len(test_labels)
test_labels

In [ ]:
# predicting the labels of the blind(test) dataset


In [ ]:
loaded_encoder_layer = EncoderLayer()
loaded_encoder_layer.load_state_dict(torch.load(best_encoder_path))
loaded_encoder_layer.eval()

# **making pytorch datasets**